<a href="https://colab.research.google.com/github/djrl10/trying/blob/master/LSTM_SDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source:
https://github.com/c2nes/javalang
Software Defect Prediction via ConvolutionalNeural Network
Seml: A Semantic LSTM Model for Software Defect Prediction
Automatically Learning Semantic Features for Defect Prediction
https://github.com/kevinqiu1990/TCNN
Transfer Convolutional Neural Network for Cross-Project Defect Prediction 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install javalang
import tensorflow as tf
import javalang
import javalang.tree as jlt
from os import walk
import pandas as pd
from io import StringIO
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np
import codecs
import os
import glob
import re , itertools
from html import unescape
!pip install unidecode
from unidecode import unidecode
import nltk
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize
from string import punctuation
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt

     |████████████████████████████████| 245kB 4.3MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


1)Method invocation and class instance creation nodes:
We extract and record these nodes using their method or
class names. For example, we extract method pop() and
push() in File2.java in Figure 1 and record them as pop
and push.

2)Declaration nodes: We extract method declaration, type
declaration and enum declaration nodes, and record them
using their names.

3)Control-flow nodes: We extract those statements or
clauses related to control flow of a program, e.g., if state-
ment, for statement, while statement, catch clause.
Control-flow nodes are recorded using their statement types,
e.g., if statement is recorded as if, and catch clause is recorded
as catch.
Therefore

karena hanya 3 aturan yg digunakan maka types node dari AST yg digunakan adalah dibawah ini:

In [3]:
types = [jlt.FormalParameter, jlt.BasicType, jlt.PackageDeclaration, jlt.InterfaceDeclaration, jlt.CatchClauseParameter,
         jlt.ClassDeclaration, jlt.MethodInvocation, jlt.SuperMethodInvocation, jlt.MemberReference, jlt.SuperMemberReference,
         jlt.ConstructorDeclaration, jlt.ReferenceType, jlt.MethodDeclaration, jlt.VariableDeclarator, jlt.IfStatement,
         jlt.WhileStatement, jlt.DoStatement, jlt.ForStatement, jlt.AssertStatement, jlt.BreakStatement,
         jlt.ContinueStatement, jlt.ReturnStatement, jlt.ThrowStatement, jlt.SynchronizedStatement, jlt.TryStatement,
         jlt.SwitchStatement, jlt.BlockStatement, jlt.StatementExpression, jlt.TryResource, jlt.CatchClause,
         jlt.CatchClauseParameter, jlt.SwitchStatementCase, jlt.ForControl, jlt.EnhancedForControl]

Function untuk parse dari source code ke AST -> Token

Didefinisikan secara manual

In [4]:
def parse(source):
    tokens = javalang.parse.parse(source)
    result = []
    for path, node in tokens:
        if isinstance(node, jlt.PackageDeclaration):
            result.append(node.name)
            continue
        if isinstance(node, jlt.IfStatement):
            result.append("if")
            continue
        if isinstance(node, jlt.InterfaceDeclaration):
            result.append(node.name)
            continue
        if isinstance(node, jlt.ClassDeclaration):
            result.append(node.name)
            continue
        if isinstance(node, jlt.MethodInvocation):
            result.append(node.member)
            continue
        if isinstance(node, jlt.MemberReference):
            result.append(node.member)
            continue
        if isinstance(node, jlt.ConstructorDeclaration):
            result.append(node.name)
            continue
        if isinstance(node, jlt.ReferenceType):
            result.append(node.name)
            continue
        if isinstance(node, jlt.MethodDeclaration):
            result.append(node.name)
            continue
        if isinstance(node, jlt.VariableDeclarator):
            result.append(node.name)
            continue
        if isinstance(node, jlt.ThrowStatement):
            result.append("throw")
            continue
        if isinstance(node, jlt.TryStatement):
            result.append("try")
            continue
        if isinstance(node, jlt.BlockStatement):
            result.append("block")
            continue
        if isinstance(node, jlt.CatchClause):
            result.append("catch")
            continue
        if isinstance(node, jlt.ForStatement):
            result.append("for")
            continue
        if isinstance(node, jlt.AssertStatement):
            result.append("assert")
            continue
        if isinstance(node, jlt.BreakStatement):
            result.append("break")
            continue
        if isinstance(node, jlt.ContinueStatement):
            result.append("cotinue")
            continue
        if isinstance(node, jlt.ReturnStatement):
            result.append("return")
            continue
        if isinstance(node, jlt.ReturnStatement):
            result.append("return")
            continue
        if isinstance(node, jlt.ThrowStatement):
            result.append("throw")
            continue
        if isinstance(node, jlt.SwitchStatement):
            result.append("switch")
            continue
        if isinstance(node, jlt.WhileStatement):
            result.append("while")
            continue
        if isinstance(node, jlt.DoStatement):
            result.append("do")
            continue
        if isinstance(node, jlt.CatchClauseParameter):
            result.append(node.types)
            continue
        if isinstance(node, jlt.BasicType):
            result.append(node.name)
            continue
        if isinstance(node, jlt.SynchronizedStatement):
            result.append("Synchronized")
            continue
    return result


Proses mengubah Source Code (File) -> AST -> Token

Arsitektur Folder


"Nama Project"

    "Defect"

        "File Defect 1"

        "File Defect 2"

        ....

    "Non-Defect"

        "File Non-Defect 1"

        "File Non-Defect 2"

        ....

In [5]:
def Tokenize(dir_name):
    df_list = []
    df = pd.DataFrame(columns=['File','Token','Defect'])
    for (dirpath, dirnames, filenames) in walk(dir_name):
        for label in dirnames: 
            for (dirpath, dirnames, filenames) in walk(dir_name+'/'+label):
                for filename in filenames:
                    f = open(dir_name+'/'+label+'/'+filename, "r")
                    f = f.read()
                    try:
                        p = parse(f)
                    except:
                        continue

                    temp = pd.DataFrame([[filename,p,label]],columns=['File','Token','Defect'])
                    df_list.append(temp)
                break
        break
    df = pd.concat(df_list)
    df = df.reset_index()
    return df

In [6]:
list_label = ['Tokencamel-1_6', 'Tokencamel-1_4']
list_path = ['/content/drive/My Drive/data_set/camel-1.6', '/content/drive/My Drive/data_set/camel-1.4']
datasets = dict()


In [ ]:
for index in range(2) :
  data = Tokenize(list_path[index])
  datasets.update({list_label[index] : data})

In [ ]:
features = dict()
labels = dict()

In [ ]:
for key in datasets.keys() :
  list_token = datasets[key].iloc[:, 2:-1]
  label = datasets[key].iloc[:, -1:].values
  features.update({key : list_token})
  labels.update({key : label})
  

In [ ]:
clean_features = dict()

In [ ]:
for key in datasets.keys() :
  feature = features[key].values
  clean_feature = list()
  for idx, val in enumerate(feature) :
    filtered_feature = [x for x in val[0] if isinstance(x, str)]
    clean_feature.append(filtered_feature)
  clean_features.update({key : clean_feature})

In [ ]:
join_features = dict()
join_label = dict()

In [ ]:
for key in datasets.keys() :
  lb = pd.DataFrame(labels[key], columns=['labels'])
  lb_dum = pd.get_dummies(lb.labels)
  lb_dum.head()
  label = lb_dum[['Defect', 'Non-Defect']].values
  join_label.update({key : label})
  new_feature = [', '.join(item) for item in clean_features[key]]
  join_features.update({key : new_feature})

#Word Embedding

In [ ]:
cv = CountVectorizer()
tfidf = TfidfTransformer()

In [ ]:
join_features[key]

In [ ]:
join_label[key]

# Split data into training and validation

In [ ]:
token_trains = dict()
token_tests = dict()
label_trains = dict()
label_tests = dict()
for key in datasets.keys():
  tkn_train, tkn_test, lbl_train, lbl_test = train_test_split(join_features[key], join_label[key], test_size=0.2, shuffle=True)
  token_trains.update({key:tkn_train})
  token_tests.update({key:tkn_test})
  label_trains.update({key:lbl_train})
  label_tests.update({key:lbl_test})

In [ ]:
print(len(token_trains['Tokencamel-1_4']))
print(len(token_tests['Tokencamel-1_4']))
print(len(label_trains['Tokencamel-1_4']))
print(len(label_tests['Tokencamel-1_4']))

In [ ]:
# tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_trains = dict()
padded_tests = dict()

for key in datasets.keys():
  tokenizer = Tokenizer(num_words=5000, oov_token='x', filters='!"#$%&()*+,-./:;<=>@[\]^_`{|}~ ')
  tokenizer.fit_on_texts(token_trains[key]) 
  tokenizer.fit_on_texts(token_tests[key])
 
  sekuens_train = tokenizer.texts_to_sequences(token_trains[key])
  sekuens_test = tokenizer.texts_to_sequences(token_tests[key])
 
  padded_train = pad_sequences(sekuens_train) 
  padded_test = pad_sequences(sekuens_test)

  padded_tests.update({key : padded_test})
  padded_trains.update({key : padded_train})

  print(padded_test.shape)
  print(padded_train.shape)

#Model LSTM

In [ ]:
# model
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=64),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='sigmoid')
])
model.compile(optimizer='adam', metrics=['accuracy'], loss='binary_crossentropy',)
model.summary()

In [ ]:
# model fit
results = dict()
for key in datasets.keys():
  history = model.fit(padded_trains[key], label_trains[key], epochs=5, 
                    steps_per_epoch = 30, validation_data=(padded_tests[key], label_tests[key]), verbose=2, 
                    validation_steps=30)
  results.update({key:history})

In [ ]:
for key in datasets.keys():  
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('Model Accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.show()

In [ ]:
# plot of loss
for key in datasets.keys(): 
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model Loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.show()

#Model RNN

In [ ]:
# model
import tensorflow as tf
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=64),
    tf.keras.layers.SimpleRNN(128),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='sigmoid')
])
model2.compile(optimizer='adam', metrics=['accuracy'], loss='binary_crossentropy',)
model2.summary()

In [ ]:
# model fit
results = dict()
for key in datasets.keys():
  history = model2.fit(padded_trains[key], label_trains[key], epochs=5, 
                    steps_per_epoch = 30, validation_data=(padded_tests[key], label_tests[key]), verbose=2, 
                    validation_steps=30)
  results.update({key:history})

In [ ]:
for key in datasets.keys():  
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('Model Accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.show()

In [ ]:
# plot of loss
for key in datasets.keys(): 
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model Loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.show()